# Import Package 

In [1]:
import re
import json
import math
import numpy as np
import collections
import xml.dom.minidom
from stemming.porter2 import stem

#  Input from .txt File into dictionary

In [2]:
# Term -> (Document -> Position)
# input term_doc_pos_dict
# {'Term1': {doc1: [position1, position2], doc2: [position1, position2]}
#  'Term2': {doc1: [position1, position2, position3], doc2: [position1, position2]}}

file = open('index_ts.txt', 'r') 
js = file.read()
term_doc_pos_dict = json.loads(js) 
file.close() 

# term_doc_pos_dict

In [3]:
# Term -> Position
# input from term_position_dict
# {'Term1': [[doc1, pos1], [doc2, pos2], [doc3, pos3]
#  'Term2': [[doc1, pos1], [doc2, pos2], [doc3, pos3]}

file = open('index_ts_pos.txt', 'r') 
js = file.read()
term_position_dict = json.loads(js) 
file.close() 

# term_position_dict

# Get Score of each Query

In [9]:
# stopping words list
with open('englishST.txt', 'r') as eng:
    eng_str = eng.read()
eng_list = eng_str.split()  

def takeSecond(elem):
    return elem[1]

def getScore(s, term_doc_pos_dict):
    s_list = s.split()
    for m in range(len(s_list)):
        s_list[m] = s_list[m].lower()  # lower()

    # Stopping 
    stop_list = []
    for i in s_list:
        if (i not in eng_list):
            stop_list.extend([i])
    
    # Normalisation
    w_list = []
    for j in stop_list:
        w_list.extend([stem(j)])
    
    print(w_list)
    
    # get document list
    doc_list = list(term_doc_pos_dict[w_list[0]].keys()) 
    for i in range(len(w_list)-1):
        doc_list = list(set(doc_list).union(term_doc_pos_dict[w_list[i+1]].keys())) 
    for j in range(len(doc_list)):
        doc_list[j] = int(doc_list[j])
    doc_list.sort()
    
    # get score list
    score_list = []
    for i in range(len(doc_list)):
        score_doc = 0
        for j in range(len(w_list)):           
            if (str(doc_list[i]) in list(term_doc_pos_dict[w_list[j]].keys())):
                a = 1 + math.log10(len(term_doc_pos_dict[w_list[j]][str(doc_list[i])]))
                # print(a)
                b = math.log10(5000/len(term_doc_pos_dict[w_list[j]].keys()))
                # print(b)
                score_doc = score_doc + a * b
        score_list.extend([[int(doc_list[i]), format(score_doc, '.4f')]])  
    
    # sort according to 1st column
    score_list.sort(key=takeSecond, reverse=True)
              
    return score_list

In [10]:
len(getScore("FT article on the BBC and BSkyB deal", term_doc_pos_dict))

['ft', 'articl', 'bbc', 'bskyb', 'deal']


5000

# Generate term_score_dict 

In [8]:
# Input query txt
with open('queries.ranked.txt', 'r') as f:
    line_list = f.readlines()
f.close()

term_score_dict = {}
num_list = []
l_list = []
r = """[0-9?]"""

# get the number
for i in range(len(line_list)):  
    num_list.extend([line_list[i].split()])  # split 
# get the line
for j in range(len(line_list)):
    l_list.extend([re.sub(r, '', line_list[j]).strip()])  # detele 0-9
print(l_list)

for k in range(len(l_list)):
    term_score_dict[int(num_list[k][0])] = getScore(l_list[k], term_doc_pos_dict)
    print(len(term_score_dict[int(num_list[k][0])]))

term_score_dict

['income tax reduction', 'stock market in Japan', 'health industry', 'the Robotics industries', 'the peace process in the middle east', 'information retrieval', 'Dow Jones industrial average stocks', 'will be there a reduction in the income taxes', 'the gold prices versus the dollar price', 'FT article on the BBC and BSkyB deal']
1267
2240
1501
1388
1039
384
2002
1267
2045
5000


{1: [[9684, '5.0743'],
  [9090, '4.4829'],
  [9652, '4.3478'],
  [9574, '4.3268'],
  [9281, '4.2160'],
  [9891, '4.0384'],
  [7844, '4.0021'],
  [6171, '4.0011'],
  [9198, '3.9796'],
  [8446, '3.9646'],
  [5690, '3.8694'],
  [6850, '3.8448'],
  [9584, '3.8105'],
  [9591, '3.8105'],
  [9616, '3.7642'],
  [9665, '3.7352'],
  [5493, '3.6808'],
  [6567, '3.6772'],
  [6915, '3.6539'],
  [5023, '3.6375'],
  [7853, '3.6375'],
  [7732, '3.6234'],
  [8871, '3.5779'],
  [8124, '3.5058'],
  [6602, '3.4850'],
  [7353, '3.4470'],
  [8564, '3.3441'],
  [8419, '3.3176'],
  [7735, '3.2894'],
  [7730, '3.2745'],
  [9069, '3.2184'],
  [8179, '3.2013'],
  [7745, '3.2000'],
  [9156, '3.1213'],
  [9394, '3.1213'],
  [9625, '3.1213'],
  [5936, '3.1156'],
  [9961, '3.1156'],
  [5014, '3.0769'],
  [6675, '3.0696'],
  [7811, '3.0696'],
  [7980, '3.0168'],
  [7553, '2.9637'],
  [6473, '2.9541'],
  [7771, '2.9316'],
  [8270, '2.9316'],
  [5660, '2.8927'],
  [8375, '2.8927'],
  [5368, '2.8756'],
  [9530, '2.8756'

# Output rank txt

In [7]:
# Output to output_rank.txt
# clear
file = open('results.ranked.txt', 'w').close()

# write
doc = open('results.ranked.txt', 'w')
for key, value in term_score_dict.items():
    if (len(value) > 150):
        for i in range(150):
            print(str(key) + "," + str(value[i][0]) + "," + str(value[i][1]), file=doc)
    else:
        for i in range(len(value)):
            print(str(key) + "," + str(value[i][0]) + "," + str(value[i][1]), file=doc)
doc.close()